In [ ]:
# Load required libraries
library(tidyverse)
library(janitor)
library(dplyr)
library(ggplot2)
library(skimr)
library(purrr)
library(lubridate)

# Source helper scripts
source("../../R/apply_factors.R")
source("../../R/analysis_helpers.R")
source("../../R/temporal_helpers.R")

# Load data
tables <- list(
  Orders  = readr::read_csv("../../data/processed/Orders.csv"),
  Returns = readr::read_csv("../../data/processed/Returns.csv"),
  People  = readr::read_csv("../../data/processed/People.csv")
)

# Apply factor transformations
tables <- apply_factors(tables)

# Extract tables
orders  <- tables$Orders
returns <- tables$Returns
people  <- tables$People

Rows: 51290 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): order_id, ship_mode, customer_name, segment, state, country, mark...
dbl   (6): sales, quantity, discount, profit, shipping_cost, year
date  (2): order_date, ship_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1173 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): order_id, market
lgl (1): returned

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 14 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): person, region

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the co

# Regional Manager Performance Overview

In [ ]:
orders_joined <- orders |>
    left_join( 
        people |> rename(regional_manager = person),
        by = "region"
    )

region_summary <- orders_joined |>
    group_by(region, regional_manager) |>
    summarise(
        total_sales = sum(sales, na.rm = TRUE),
        total_profit = sum(profit, na.rm = TRUE),
        total_orders = n(),
        profit_margin = total_profit / total_sales,
        .groups = "drop"
    )

region_summary_clean <- region_summary |>
    mutate(
        regional_manager = ifelse(
            is.na(regional_manager) | regional_manager == "",
            "Unassigned",
            regional_manager
        )
    )

region_summary_clean


region,regional_manager,total_sales,total_profit,total_orders,profit_margin
<fct>,<chr>,<dbl>,<dbl>,<int>,<dbl>
Africa,Deborah Brumfield,783773.21,88871.63,4587,0.11338947
Canada,Nicole Hansen,66928.17,17817.39,384,0.26621660
Caribbean,Giulietta Dortch,324280.86,34571.32,1690,0.10660919
Central,Anna Andreadi,2822302.52,311403.98,11117,0.11033685
Central Asia,Nora Preis,752826.57,132480.19,2048,0.17597703
East,Kelly Williams,678781.24,91522.78,2848,0.13483399
EMEA,Unassigned,806161.31,43897.97,5029,0.05445309
North,Jack Lebron,1248165.60,194597.95,4785,0.15590716
North Asia,Shirley Daniels,848309.78,165578.42,2338,0.19518627
